In [1]:
import pandas as pd, numpy as np
from rolling_framework import Machine

import warnings
warnings.filterwarnings('ignore')

/Users/ethan_hong/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [2]:
period = '19712023'

#%% Target variables : excess returns
y = pd.read_csv(f'data/{period}/exrets_{period}.csv', index_col='Time')             # Target : Excess Return (Bond Risk Premia)
y = y[['xr_2','xr_5','xr_7','xr_10']]

#%% Predictors (Features)
FWDS = pd.read_csv(f'data/{period}/fwds_{period}.csv', index_col='Time')            # Forward Rates
MACV = pd.read_csv(f'data/{period}/MacroFactors_{period}.csv', index_col='Time')    # Ludvigson and Ng (2009), Macrovariables
LSC = pd.read_csv(f'data/{period}/lsc_{period}.csv', index_col='Time')              # Level, Slope and Curvature
# YLV = pd.read_csv(f'data/{period}/yl_{period}.csv', index_col='Time')               # yield-level
RVOL = pd.read_csv(f'data/{period}/real_vol_{period}.csv', index_col='Time')        # Realised Volatility 10yr
#IVOL = pd.read_csv(f'data/{period}/imp_vol_{period}.csv', index_col='Time')        # Implied Volatility (Work for 1990~)
CP = pd.read_csv(f'data/{period}/cp_{period}.csv', index_col='Time')                # Cochrane-Piazessi Factor
# YALL = pd.read_csv(f'data/{period}/yl_all_{period}.csv', index_col='Time')          # 12 m, 24 m, ... 120 m
# YMAX = pd.read_csv(f'data/{period}/yl_max_{period}.csv', index_col='Time')          # 1m ... 120 m

F6 = MACV[['F1','F2','F3','F4','F8','F1^3']]
SL = LSC[['slope']]
LV = LSC[['level']]
CU = LSC[['curvature']]


AGG = pd.concat([CP, FWDS], axis=1)

In [12]:
# ────────────────────────────
# Random-Forest
# ────────────────────────────
param_grid_rf = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5,1],
}

# ────────────────────────────
# Extra-Trees
# ────────────────────────────
param_grid_et = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5, 1],
}

# ────────────────────────────
# XGBoost  (XGBRegressor 사용 가정)
# ────────────────────────────
param_grid_xgb = {
    "model__estimator__n_estimators":  [300],
    "model__estimator__max_depth":     [2, 4],
    "model__estimator__learning_rate": [0.01],
    "model__estimator__subsample":     [0.7, 0.5],
    "model__estimator__reg_lambda":    [0.1, 1.0],
}

param_grid_lasso = {'reg__alpha': [0.001, 1.0]}    
param_grid_ridge      = {'reg__alpha': [0.001, 1]}
param_grid_elasticnet = {
    'reg__alpha':   [0.01, 0.1, 1, 10],
    'reg__l1_ratio':[0.1, 0.3, 0.5],
}

### Helper

In [4]:
# ────────────────────────────
# Custom Weights for Portfolio
# ────────────────────────────
w_custom = pd.Series(
    {
        "xr_2" : 2,   # 2Y 듀레이션
        "xr_5" : 5,   # 5Y 듀레이션
        "xr_7" : 7,   # 7Y 듀레이션
        "xr_10": 10,   # 10Y 듀레이션
    },
    name="w"
)

cols_dw = w_custom.index.tolist()
# 합이 1이 되도록 정규화
w_custom = w_custom / w_custom.sum()


# ────────────────────────────
bs = "197108" # burn-in-start
be = "199101" # burn-in-end
p = ["197108", "202312"] #period

### OLS, LASSO, ELASTICNET

In [5]:
ols_1 = Machine(SL, y, 'OLS' , burn_in_start=bs, burn_in_end=be, period=p)
ols_1.training()
ols_2 = Machine(CP,y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_2.training()
ols_3 = Machine(F6, y, 'OLS', burn_in_start=bs, burn_in_end=be, period=p)
ols_3.training()

print(ols_1.R2OOS())
print('ew :', ols_1.r2_oos_portfolio())
print('dw :', ols_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_2.R2OOS())
print('ew :', ols_2.r2_oos_portfolio())
print('dw :', ols_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_3.R2OOS())
print('ew :', ols_3.r2_oos_portfolio())
print('dw :', ols_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

OLS rolling: 100%|██████████| 396/396 [00:15<00:00, 25.57it/s]


xr_2    -0.082473
xr_5     0.003289
xr_7     0.042446
xr_10    0.105551
dtype: float64
ew : 0.008501604412247121
dw : 0.020249067203341076
xr_2    -0.023877
xr_5     0.011705
xr_7     0.013968
xr_10    0.027867
dtype: float64
ew : -0.025039860544500137
dw : -0.029797681402277565
xr_2    -0.256994
xr_5    -0.026228
xr_7     0.005076
xr_10    0.060933
dtype: float64
ew : -0.03500152715958782
dw : -0.021103619826791098


In [6]:
lasso_1 = Machine(SL, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_1.training()
lasso_2 = Machine(CP, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_2.training()
lasso_3 = Machine(F6, y, 'Penalized', option='lasso', params_grid=param_grid_lasso, burn_in_start=bs, burn_in_end=be, period=p)
lasso_3.training()


print(lasso_1.R2OOS())
print('ew :', lasso_1.r2_oos_portfolio())
print('dw :', lasso_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_2.R2OOS())
print('ew :', lasso_2.r2_oos_portfolio())
print('dw :', lasso_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_3.R2OOS())
print('ew :', lasso_3.r2_oos_portfolio())
print('dw :', lasso_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 396/396 [00:37<00:00, 10.65it/s]


xr_2    -0.149287
xr_5    -0.065618
xr_7     0.013893
xr_10    0.096883
dtype: float64
ew : -0.022049770814446035
dw : -0.0023632065197169982
xr_2    -0.001688
xr_5     0.035842
xr_7     0.041587
xr_10    0.053626
dtype: float64
ew : 0.009265357406012309
dw : 0.0011704107817599452
xr_2    -0.319276
xr_5    -0.067205
xr_7    -0.010821
xr_10    0.043627
dtype: float64
ew : -0.061095781907104074
dw : -0.043538287447144386


In [10]:
ridge_1 = Machine(SL, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_1.training()
ridge_2 = Machine(CP, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_2.training()
ridge_3 = Machine(F6, y, 'Penalized', option='ridge', params_grid=param_grid_ridge, burn_in_start=bs, burn_in_end=be, period=p)
ridge_3.training()


print(ridge_1.R2OOS())
print('ew :', ridge_1.r2_oos_portfolio())
print('dw :', ridge_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_2.R2OOS())
print('ew :', ridge_2.r2_oos_portfolio())
print('dw :', ridge_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_3.R2OOS())
print('ew :', ridge_3.r2_oos_portfolio())
print('dw :', ridge_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 396/396 [00:06<00:00, 56.84it/s]


xr_2    -0.082582
xr_5     0.003173
xr_7     0.042387
xr_10    0.105558
dtype: float64
ew : 0.008435473548068617
dw : 0.02020691512208106
xr_2    -0.023410
xr_5     0.012221
xr_7     0.014598
xr_10    0.028551
dtype: float64
ew : -0.024372116246686115
dw : -0.029108893140944225
xr_2    -0.254703
xr_5    -0.025088
xr_7     0.006191
xr_10    0.061808
dtype: float64
ew : -0.033834996974841935
dw : -0.020031664135687466


In [7]:
elasticnet_1 = Machine(SL, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_1.training()

elasticnet_2 = Machine(CP, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_2.training()

elasticnet_3 = Machine(F6, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_3.training()


print(elasticnet_1.R2OOS())
print('ew :', elasticnet_1.r2_oos_portfolio())
print('dw :', elasticnet_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_2.R2OOS())
print('ew :', elasticnet_2.r2_oos_portfolio())
print('dw :', elasticnet_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_3.R2OOS())
print('ew :', elasticnet_3.r2_oos_portfolio())
print('dw :', elasticnet_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling:   0%|          | 0/360 [00:00<?, ?it/s]

Penalized rolling: 100%|██████████| 360/360 [01:12<00:00,  5.00it/s]


xr_2    -0.175665
xr_5    -0.089630
xr_7    -0.028150
xr_10    0.063809
dtype: float64
ew : -0.1658429596938562
dw : -0.14602176236112752
xr_2    -0.042565
xr_5     0.024434
xr_7     0.054527
xr_10    0.099454
dtype: float64
ew : -0.06800728526477418
dw : -0.0670683359962545
xr_2    -0.234712
xr_5    -0.089215
xr_7    -0.054463
xr_10   -0.005559
dtype: float64
ew : -0.20162131707054187
dw : -0.19458831277632793


### Tree

In [8]:
rf_1 = Machine(SL, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_1.training()
rf_2 = Machine(CP, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_2.training()
rf_3 = Machine(F6, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf, burn_in_start=bs, burn_in_end=be, period=p)
rf_3.training()

print(rf_1.R2OOS())
print('ew :', rf_1.r2_oos_portfolio())
print('dw :', rf_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_2.R2OOS())
print('ew :', rf_2.r2_oos_portfolio())
print('dw :', rf_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_3.R2OOS())
print('ew :', rf_3.r2_oos_portfolio())
print('dw :', rf_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 396/396 [1:10:16<00:00, 10.65s/it]


xr_2    -0.230680
xr_5    -0.138054
xr_7    -0.095965
xr_10   -0.050871
dtype: float64
ew : -0.1438066592432523
dw : -0.1356678879449329
xr_2    -0.110794
xr_5    -0.106382
xr_7    -0.122055
xr_10   -0.129678
dtype: float64
ew : -0.17064285043956184
dw : -0.1829130869812936
xr_2    -0.079444
xr_5    -0.008482
xr_7     0.002558
xr_10    0.032217
dtype: float64
ew : -0.0329908243919006
dw : -0.033061157755871484


In [9]:
et_1 = Machine(SL, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_1.training()
et_2 = Machine(CP, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_2.training()
et_3 = Machine(F6, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et, burn_in_start=bs, burn_in_end=be, period=p)
et_3.training()


print(et_1.R2OOS())
print('ew :', et_1.r2_oos_portfolio()) 
print('dw :', et_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_2.R2OOS())
print('ew :', et_2.r2_oos_portfolio())
print('dw :', et_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_3.R2OOS())
print('ew :', et_3.r2_oos_portfolio())
print('dw :', et_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


Tree rolling: 100%|██████████| 396/396 [31:45<00:00,  4.81s/it]


xr_2    -0.187796
xr_5    -0.091371
xr_7    -0.033015
xr_10    0.043922
dtype: float64
ew : -0.07246599002427523
dw : -0.05517140707816215
xr_2    -0.011248
xr_5     0.011554
xr_7     0.021166
xr_10    0.038569
dtype: float64
ew : -0.017368861694225357
dw : -0.021291396022271147
xr_2    -0.087882
xr_5    -0.004555
xr_7     0.011782
xr_10    0.029876
dtype: float64
ew : -0.031556297175501236
dw : -0.03183203777760846


In [13]:
xgb_1 = Machine(SL, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_1.training()
xgb_2 = Machine(CP, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_2.training()
xgb_3 = Machine(F6, y, 'Tree', option='XGB', params_grid=param_grid_xgb, burn_in_start=bs, burn_in_end=be, period=p)
xgb_3.training()

print(xgb_1.R2OOS())
print('ew :', xgb_1.r2_oos_portfolio())
print('dw :', xgb_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_2.R2OOS())
print('ew :', xgb_2.r2_oos_portfolio())
print('dw :', xgb_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_3.R2OOS())
print('ew :', xgb_3.r2_oos_portfolio())
print('dw :', xgb_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 396/396 [03:08<00:00,  2.10it/s]


xr_2    -0.217348
xr_5    -0.150001
xr_7    -0.092922
xr_10   -0.025257
dtype: float64
ew : -0.1351570379708873
dw : -0.1219819760847689
xr_2    -0.078179
xr_5    -0.053915
xr_7    -0.061887
xr_10   -0.051353
dtype: float64
ew : -0.1024862664568924
dw : -0.11004405088764835
xr_2    -0.084858
xr_5    -0.002334
xr_7     0.004593
xr_10    0.033728
dtype: float64
ew : -0.031301440924228485
dw : -0.030854654699884243


### DNN : FWDs ONly

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_1 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_1.training()
print(dnn_1.R2OOS())
print('ew :', dnn_1.r2_oos_portfolio())
print('dw :', dnn_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


### DNN : FWDS + MACRO

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,3)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_2 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12,
    random_state=15
)
dnn_2.training()
print(dnn_2.R2OOS())
print('ew :', dnn_2.r2_oos_portfolio())
print('dw :', dnn_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


In [ ]:
AGG = pd.concat([MACV, CP], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(1)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG.columns if c.startswith("F")]
grp2_cols = [c for c in AGG.columns if c not in grp1_cols]

dnn_d1 = Machine(
    AGG, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    params_grid=param_grid,
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    forecast_horizon=12
)
dnn_d1.training()
print(dnn_d1.R2OOS())

In [ ]:
AGG2 = pd.concat([MACV, FWDS], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(3,3)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG2.columns if c.startswith("F")]
grp2_cols = [c for c in AGG2.columns if c not in grp1_cols]

dnn_d2 = Machine(
    AGG2, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    burn_in_start=bs,
    burn_in_end=be,
    period=p,
    params_grid=param_grid,
    forecast_horizon=12
)
dnn_d2.training()
print(dnn_d2.R2OOS())